In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import json
import numpy as np

# Load the pre-trained model
model_config = '/content/drive/MyDrive/yolov4/darknet/cfg/yolov4-custom.cfg'
model_weights = '/content/drive/MyDrive/yolov4/darknet/yolov4.conv.137'
net = cv2.dnn.readNetFromDarknet(model_config, model_weights)

# Define the class labels
class_file = '/content/drive/MyDrive/yolov4/darknet/data/obj.names'
with open(class_file, 'r') as f:
    class_labels = f.read().strip().split('\n')

# Define the colors for the bounding boxes
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(class_labels), 3), dtype='uint8')

# Open the video file
video_path = '/content/drive/MyDrive/yolov4/mask_test_videos/026c7465-309f6d33.mp4'
cap = cv2.VideoCapture(video_path)

In [9]:
print(net)

< cv2.dnn.Net 0x7f59e2b622b0>


In [8]:
while True:
    # Read the next frame from the video
    _, frame = cap.read()

    if frame is None:
        break

    # Prepare the frame for the model
    blob = cv2.dnn.blobFromImage(frame, size=(416, 16), ddepth=cv2.CV_8U)
    net.setInput(blob)

    # Perform object detection
    out = net.forward()

    # Parse the model's output
    for i in range(out.shape[0]):
        class_id = int(out[i, 0])
        confidence = float(out[i, 1])
        if confidence > 0.5:
            xmin, ymin, xmax, ymax = (out[i, 2:] * [frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]).astype(int)

            if class_id < len(class_labels):
                # Draw the bounding box on the frame
                color = colors[class_id]
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
                cv2.putText(frame, class_labels[class_id], (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Save the object's data to a JSON file
                obj_data = {
                    'class': class_labels[class_id],
                    'xmin': xmin,
                    'ymin': ymin,
                    'xmax': xmax,
                    'ymax': ymax
                }
                json_file = 'path/to/obj_data.json'
                with open(json_file, 'w') as f:
                    json.dump(obj_data, f)

    # Display the frame
    cv2.imshow('Frame', frame)


error: ignored